In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
#! /usr/bin/python3
import argparse
import logging
import os
import sys
from collections import namedtuple

import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm

#
from transformers import BertTokenizer
#from transformers import BertAbs



I0109 20:19:46.771435 139724129376064 file_utils.py:35] PyTorch version 1.3.0 available.


In [60]:
!echo $PYTHONPATH

In [62]:
!export PYTHONPATH="${PYTHONPATH}:/dadendev/Presumm/src"

In [71]:
sys.path.insert(0, "/dadendev")
sys.path.insert(0, "/dadendev/Presumm/src")
sys.path.insert(0, "/dadendev/PreSumm/src/models")

In [72]:
import loss

ModuleNotFoundError: No module named 'models.reporter'

In [4]:
sys.path.insert(0, "./transformers/examples/summarization")
from modeling_bertabs import BertAbs, build_predictor

from utils_summarization import (
    #SummarizationDataset,
    build_mask,
    compute_token_type_ids,
    encode_for_summarization,
    fit_to_block_size,
)
from run_summarization import format_summary

In [5]:
nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
from utils_nlp.models.transformers.extractive_summarization import Bunch
args = Bunch({"block_trigram":True, "alpha": 0.95, "beam_size": 5, "min_length": 20, "max_length": 200})

In [6]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.common.pytorch_utils import get_device
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

import numpy as np
import pandas as pd
import scrapbook as sb

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#from utils_nlp.models.transformers.datasets import SummarizationDataset
from utils_nlp.dataset.cnndm import CNNDMAbsSumDataset, CNNDMSummarizationDataset
#def build_data_iterator(args, tokenizer):


In [8]:
DATA_PATH = '/tmp/tmp_p_2ej2b'

In [9]:
#DATA_PATH = '/tmp/tmpsh6mbj3g'
# DATA_PATH = '/tmp/tmp1j5pb8nn'

In [10]:
QUICK_RUN = False
# the data path used to save the downloaded data file
#DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 4
CHUNK_SIZE=200
if not QUICK_RUN:
    TOP_N = -1
    CHUNK_SIZE = 2000

In [11]:
from torch.utils.data import Dataset
class SummarizationNonIterableDataset(Dataset):
    def __init__(self, source, target=None):
        self.source = source
        self.target = target
    def __len__(self):
        return len(self.source)
    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

In [12]:
train_dataset, test_dataset = CNNDMAbsSumDataset(top_n=TOP_N, local_cache_path=DATA_PATH)

I0109 20:19:50.716979 139724129376064 utils.py:173] Opening tar file /tmp/tmp_p_2ej2b/cnndm.tar.gz.
I0109 20:19:50.718289 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/test.txt.src already extracted.
I0109 20:19:51.011142 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/test.txt.tgt.tagged already extracted.
I0109 20:19:51.037797 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/train.txt.src already extracted.
I0109 20:19:58.616194 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/train.txt.tgt.tagged already extracted.
I0109 20:19:59.249289 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/val.txt.src already extracted.
I0109 20:19:59.584239 139724129376064 utils.py:181] /tmp/tmp_p_2ej2b/val.txt.tgt.tagged already extracted.


In [13]:
data = list(test_dataset.get_source()), list(test_dataset.get_target())
test_sum_dataset = SummarizationNonIterableDataset(data[0], data[1])

In [14]:
len(data[1])

11490

In [15]:
data = list(train_dataset.get_source()), list(train_dataset.get_target())
train_sum_dataset = SummarizationNonIterableDataset(data[0], data[1])

In [16]:
from collections import namedtuple
Batch = namedtuple("Batch", [ "batch_size", "src", "segs", "mask_src", "tgt", "tgt_str"])

In [17]:
def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.
    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    #print(data)
    #names = [name for name, _, _ in data]
    # summaries = [" ".join(summary_list) for _, _, summary_list in data]
    summaries = [" ".join(summary_list) for _, summary_list in data]
  

    encoded_text = [encode_for_summarization(story, summary, tokenizer) for story, summary in data]
    
    
    #""""""
    encoded_stories = torch.tensor(
        [fit_to_block_size(story, block_size, tokenizer.pad_token_id) for story, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)
    #"""
    encoded_summaries= torch.tensor(
        [fit_to_block_size(summary, block_size, tokenizer.pad_token_id) for _, summary in encoded_text]
    )
    
    print(len(encoded_stories))

    batch = Batch(
        #document_names=None,
        batch_size=len(encoded_stories),
        src=encoded_stories.to(device),
        segs=encoder_token_type_ids.to(device),
        mask_src=encoder_mask.to(device),
        tgt=encoded_summaries.to(device),
        tgt_str=summaries,
    )
    return batch

In [18]:
def encode_for_summarization(story_lines, summary_lines, tokenizer, max_len=512):
    """ Encode the story and summary lines, and join them
    as specified in [1] by using `[SEP] [CLS]` tokens to separate
    sentences.
    """
    story_lines_token_ids = [tokenizer.encode(line, max_length=max_len) for line in story_lines]
    story_token_ids = [token for sentence in story_lines_token_ids for token in sentence]
    summary_lines_token_ids = [tokenizer.encode(line, max_length=max_len) for line in summary_lines]
    summary_token_ids = [token for sentence in summary_lines_token_ids for token in sentence]

    return story_token_ids, summary_token_ids

In [19]:
from torch.utils.data import SequentialSampler, RandomSampler, DataLoader
def build_data_iterator(dataset, tokenizer, batch_size=16, device='cuda'):
   
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=device)

    iterator = DataLoader(dataset, sampler=sampler, batch_size=batch_size, collate_fn=collate_fn,)

    return iterator

In [20]:
from utils_nlp.common.pytorch_utils import get_device
device, num_gpus = get_device(num_gpus=4, local_rank=-1)

In [21]:
device

device(type='cuda')

In [22]:
import functools
from modeling_bertabs import BertSumOptimizer

In [23]:
model_name_or_path = "bert-base-uncased"

In [24]:
from transformers import BertConfig, BertModel, PreTrainedModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained(model_name_or_path, do_lower_case=True)

I0109 20:20:31.587527 139724129376064 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/daden/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [25]:
data_iterator = build_data_iterator(train_sum_dataset, tokenizer, batch_size=64, device=device)

In [26]:
args= Bunch({ "max_pos": 512,
            "dec_layers": 6,
            "dec_hidden_size": 768,
            "dec_heads": 8,
            "dec_ff_size": 2048,
            "dec_dropout": 0.2,
})
#model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
model = BertAbs(args)

I0109 20:20:31.829955 139724129376064 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/daden/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I0109 20:20:31.830872 139724129376064 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
 

vocab_size is 30522


In [27]:
model = model.to(device)

In [47]:
loss = loss.abs_loss(model.generator, symbols, model.vocab_size, device, train=True,
                          label_smoothing=args.label_smoothing)

AttributeError: module 'models' has no attribute 'loss'

In [28]:
# Prepare the optimizer
lr = {"encoder": 0.002, "decoder": 0.2}
warmup_steps = {"encoder": 20000, "decoder": 10000}
optimizer = BertSumOptimizer(model, lr, warmup_steps)

In [29]:
from tqdm import trange

In [39]:
def train(
    train_dataset,
    model,
    tokenizer,
    device,
    per_gpu_train_batch_size=4,
    n_gpu=1,
    max_steps=1e5,
    optimizer=None,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    max_grad_norm=0
):
    """ Fine-tune the pretrained model on the corpus. """
    # Load the data
    train_batch_size = per_gpu_train_batch_size * max(1, n_gpu)

    train_sampler = RandomSampler(train_dataset)
    #collate(data, tokenizer, block_size=512, device=device)
    model_collate_fn = functools.partial(collate, tokenizer=tokenizer, block_size=512, device=device)
    train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=model_collate_fn,
    )

    # Training schedule
    if max_steps > 0:
        num_train_epochs = max_steps // (
            len(train_dataloader) // gradient_accumulation_steps + 1
        )
    else:
        max_steps = (
            len(train_dataloader) // gradient_accumulation_steps * num_train_epochs
        )

    model.zero_grad()
    train_iterator = trange(int(num_train_epochs), desc="Epoch", disable=True)

    global_step = 0
    tr_loss = 0.0
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        for step, batch in enumerate(epoch_iterator):
            print("step:", step)
            source = batch.src
            target = batch.tgt
            encoder_token_type_ids = batch.segs
            encoder_mask = batch.mask_src
           
            model.train()
            outputs = model(
                source,
                target,
                encoder_token_type_ids,
                encoder_mask
            )

            loss = outputs[0]
            
            if num_gpus >1:
                loss = loss.mean()
                print(loss)
            else:
                print(loss)
            if gradient_accumulation_steps > 1:
                loss /= gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % gradient_accumulation_steps == 0:
                #torch.nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
                optimizer.step()
                model.zero_grad()
                global_step += 1

            if max_steps > 0 and global_step > max_steps:
                epoch_iterator.close()
                break

        if max_steps > 0 and global_step > max_steps:
            train_iterator.close()
            break

    return global_step, tr_loss / global_step

In [31]:
device

device(type='cuda')

In [40]:
   train(
    train_sum_dataset,
    model,
    tokenizer,
    device,
    per_gpu_train_batch_size=4,
    n_gpu=1,
    max_steps=1e5,
    optimizer=optimizer,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    max_grad_norm=0
   )

4
step: 0
tensor(-1.0089e-08, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 1
tensor(-1.2728e-08, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 2
tensor(-5.5840e-05, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 3
tensor(-5.5948e-05, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 4
tensor(-0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 5
tensor(-0.0001, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 6
tensor(-0.0002, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 7
tensor(-0.0002, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 8
tensor(-0.0004, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 9
tensor(-0.0004, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 10
tensor(-0.0005, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 11
tensor(-0.0005, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 12
tensor(-0.0007, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 13
tensor(-0.0007, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 14
tensor(-0.0008, d

4
step: 119
tensor(-0.0192, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 120
tensor(-0.0196, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 121
tensor(-0.0196, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 122
tensor(-0.0201, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 123
tensor(-0.0201, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 124
tensor(-0.0205, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 125
tensor(-0.0205, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 126
tensor(-0.0210, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 127
tensor(-0.0210, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 128
tensor(-0.0215, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 129
tensor(-0.0215, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 130
tensor(-0.0219, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 131
tensor(-0.0219, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 132
tensor(-0.0224, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 133
tensor(-

4
step: 237
tensor(-0.0495, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 238
tensor(-0.0501, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 239
tensor(-0.0501, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 240
tensor(-0.0507, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 241
tensor(-0.0507, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 242
tensor(-0.0512, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 243
tensor(-0.0512, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 244
tensor(-0.0518, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 245
tensor(-0.0518, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 246
tensor(-0.0523, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 247
tensor(-0.0523, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 248
tensor(-0.0529, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 249
tensor(-0.0529, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 250
tensor(-0.0535, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 251
tensor(-

4
step: 355
tensor(-0.0840, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 356
tensor(-0.0846, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 357
tensor(-0.0846, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 358
tensor(-0.0852, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 359
tensor(-0.0852, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 360
tensor(-0.0858, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 361
tensor(-0.0858, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 362
tensor(-0.0864, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 363
tensor(-0.0864, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 364
tensor(-0.0870, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 365
tensor(-0.0870, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 366
tensor(-0.0876, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 367
tensor(-0.0876, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 368
tensor(-0.0882, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 369
tensor(-

4
step: 473
tensor(-0.1203, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 474
tensor(-0.1209, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 475
tensor(-0.1209, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 476
tensor(-0.1215, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 477
tensor(-0.1215, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 478
tensor(-0.1221, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 479
tensor(-0.1221, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 480
tensor(-0.1228, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 481
tensor(-0.1228, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 482
tensor(-0.1234, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 483
tensor(-0.1234, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 484
tensor(-0.1240, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 485
tensor(-0.1240, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 486
tensor(-0.1246, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 487
tensor(-

...

4
step: 15440
tensor(-14.5451, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15441
tensor(-14.5447, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15442
tensor(-14.5484, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15443
tensor(-14.5478, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15444
tensor(-14.5515, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15445
tensor(-14.5519, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15446
tensor(-14.5549, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15447
tensor(-14.5554, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15448
tensor(-14.5586, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15449
tensor(-14.5575, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15450
tensor(-14.5619, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15451
tensor(-14.5618, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15452
tensor(-14.5656, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15453
tensor(-14.5649, device='cuda:0', gra

4
step: 15553
tensor(-14.7358, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15554
tensor(-14.7384, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15555
tensor(-14.7395, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15556
tensor(-14.7427, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15557
tensor(-14.7430, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15558
tensor(-14.7461, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15559
tensor(-14.7463, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15560
tensor(-14.7485, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15561
tensor(-14.7496, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15562
tensor(-14.7518, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15563
tensor(-14.7530, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15564
tensor(-14.7559, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15565
tensor(-14.7561, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15566
tensor(-14.7602, device='cuda:0', gra

4
step: 15666
tensor(-14.9319, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15667
tensor(-14.9317, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15668
tensor(-14.9352, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15669
tensor(-14.9355, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15670
tensor(-14.9385, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15671
tensor(-14.9385, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15672
tensor(-14.9421, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15673
tensor(-14.9418, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15674
tensor(-14.9451, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15675
tensor(-14.9452, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15676
tensor(-14.9490, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15677
tensor(-14.9491, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15678
tensor(-14.9524, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15679
tensor(-14.9525, device='cuda:0', gra

4
step: 15779
tensor(-15.1249, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15780
tensor(-15.1289, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15781
tensor(-15.1289, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15782
tensor(-15.1320, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15783
tensor(-15.1320, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15784
tensor(-15.1359, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15785
tensor(-15.1360, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15786
tensor(-15.1391, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15787
tensor(-15.1387, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15788
tensor(-15.1428, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15789
tensor(-15.1426, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15790
tensor(-15.1455, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15791
tensor(-15.1463, device='cuda:0', grad_fn=<MeanBackward0>)
4
step: 15792
tensor(-15.1494, device='cuda:0', gra

KeyboardInterrupt: 

In [ ]:
generated_summaries[0]

In [ ]:
data[1][0]

In [ ]:
reference_summaries = []
for i in data[1]:
    reference_summaries.append(i[0].replace('<q>', ''))

In [ ]:
   def _write_list_to_file(list_items, filename):
        with open(filename, "w") as filehandle:
            # for cnt, line in enumerate(filehandle):
            for item in list_items:
                filehandle.write("%s\n" % item)

In [ ]:
#_write_list_to_file(generated_summaries, "./generated_summaries")

In [ ]:
generated_summaries = []
with open("./generated_summaries", "r") as filehandle:
    for cnt, line in enumerate(filehandle):
        generated_summaries.append(line)

In [ ]:
len(generated_summaries)

In [ ]:
generated_summaries[0]

In [ ]:
reference_summaries[0]

In [ ]:
import rouge
import nltk

nltk.download("punkt")
rouge_evaluator = rouge.Rouge(
    metrics=["rouge-n", "rouge-l"],
    max_n=2,
    limit_length=True,
    length_limit=args.beam_size,
    length_limit_type="words",
    apply_avg=True,
    apply_best=False,
    alpha=0.5,  # Default F1_score
    weight_factor=1.2,
    stemming=True,
)

In [ ]:
#scores = rouge_evaluator.get_scores(generated_summaries, reference_summaries)
#str_scores = format_rouge_scores(scores)

In [ ]:
#scores

In [ ]:
#str_scores